In [1]:
chr(0)

'\x00'

In [2]:
print(chr(0))

 


In [3]:
print(chr(0).__repr__())

'\x00'


In [4]:
test_string = "this is a test" + chr(0) + "string"

In [5]:
print(test_string)

this is a test string


In [1]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")

In [7]:
print(utf8_encoded)
print(list(utf8_encoded))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33]


In [8]:
len(test_string), len(utf8_encoded)

(13, 23)

In [9]:
utf8_encoded.decode("utf-8")

'hello! こんにちは!'

In [10]:
utf16_encoded = test_string.encode("utf-16")
utf32_encoded = test_string.encode("utf-32")

In [11]:
print (len(list(utf8_encoded)))
print (len(list(utf16_encoded)))
print (len(list(utf32_encoded)))

23
28
56


In [12]:
test_string = "hello"
print(len(test_string))
print(len(list(test_string.encode("utf-8"))))

5
5


In [13]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
	return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [14]:
decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

'hello'

In [18]:
list("hello".encode("utf-8"))

[104, 101, 108, 108, 111]

In [ ]:
# decode_utf8_bytes_to_str_wrong("hello! こんにちは!".encode("utf-8")) # Error

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

In [16]:
bytes(list("こ".encode("utf-8"))[:-1]).decode("utf-8") # Error - probably no byte sequence can be prefix of any other?

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data

In [54]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [46]:
import regex as re

In [47]:
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [48]:
re.findall(PAT, "this is a piece of random text")

['this', ' is', ' a', ' piece', ' of', ' random', ' text']

In [49]:
re.findall(PAT, "this is a piece of random text.")

['this', ' is', ' a', ' piece', ' of', ' random', ' text', '.']

In [ ]:
corpus = []
with open('tiny_corpus.txt', 'r') as f:
	for line in f.readlines():
		corpus.extend([word.strip() for word in line.split(' ')])

In [82]:
from collections import Counter

def bpe_tokenizer(corpus, num_merges):
	new_tokens = []

	# Create a dict mapping pretokens to frequency
	freq = Counter()

	for pretoken in corpus:
		freq[tuple(pretoken)] += 1

	# Loop through for num_merges
	for _ in range(num_merges):

		# Count the number of co-orccurences
		bigrams = Counter()

		for pretoken in freq.keys():
			for char_1, char_2 in zip(pretoken[:-1], pretoken[1:]):
				bigrams[(char_1, char_2)] += freq[pretoken]

		# Get the max one resolving ties lexicographically
		max_val = max(bigrams.values())
		max_bigrams = [bigram for bigram in bigrams if bigrams[bigram] == max_val ]

		# Append to new_tokens
		merge_rule = max(max_bigrams)
		new_tokens.append(''.join(merge_rule))

		# Create new dict mapping pretokens to frequency with merge
		new_freq = {}
		for pretoken, val in freq.items():
			i = 0
			new_pretoken = []
			while i < len(pretoken) - 1:
				char_1 = pretoken[i]
				char_2 = pretoken[i + 1]

				if char_1 == merge_rule[0] and char_2 == merge_rule[1]:
					new_pretoken.append(char_1 + char_2)
					i += 2
				else:
					new_pretoken.append(char_1)
					i += 1

			if i == len(pretoken) - 1:
				new_pretoken.append(pretoken[i])
			new_freq[tuple(new_pretoken)] = val
		
		# Replace existing mapping
		freq = new_freq
	return new_tokens

In [ ]:
bpe_tokenizer(corpus, 6)

# Making an assumption here that each character is mapping to 1 byte

['st', 'est', 'ow', 'low', 'west', 'ne']

In [1]:
with open('data/TinyStoriesV2-GPT4-train.txt', 'r') as f:
	lines = f.readlines()

In [2]:
len(lines[0])

1

#### GRAVEYARD

In [33]:
from cs336_basics.pretokenization_example import find_chunk_boundaries
f = open('data/TinyStoriesV2-GPT4-train.txt', 'rb')
desired_num_chunks = 100
split_special_token = b'<|endoftext|>'
chunk_boundaries = find_chunk_boundaries(f, desired_num_chunks, split_special_token)

In [34]:
i = 0
f.seek(chunk_boundaries[i])
op = f.read(chunk_boundaries[i + 1] - chunk_boundaries[i])

In [5]:
op_str = op.decode('utf-8')

In [6]:
print(op_str[-2000:])

d her mom went to the store and picked out the most beautiful kite they could find. It was red, blue, and yellow. Lily was so grateful to her mom for helping her get the kite. They went to the park to fly the kite together.
At the park, Lily and her mom ran and laughed as they flew the kite high in the sky. The kite danced in the wind, and Lily's dream had come true. She was so grateful for her mom and the fun day they had together.
From that day on, Lily and her mom flew the kite at the park whenever they could. They had so much fun together, and Lily knew that dreams can come true. She was very grateful for her mom and her pretty, colorful kite.
<|endoftext|>
Once upon a time, there was a pretty landscape with many trees. It was a happy place. One day, a big wind came and blew very hard. It made the trees bend and shake. The animals in the landscape were scared.
A little bird had a nest in one of the trees. The bird's nest started to hang on a branch, about to fall. The little bird w

In [7]:
print(op_str[:1000])


Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  
He said, “Wow, that is a really amazing vase! Can I buy it?” 
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. 
And that's how Ben found an amazing vase in the store!
<|endoftext|>
Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his fri

In [9]:
split_special_token.decode('utf-8')

'<|endoftext|>'

In [45]:
import regex as re
import multiprocessing as mp
from collections import Counter
from cs336_basics.pretokenization_example import find_chunk_boundaries

file = open('data/TinyStoriesV2-GPT4-train.txt', 'rb')
desired_num_chunks = 100
split_special_token = b'<|endoftext|>'
special_tokens = [split_special_token]

chunk_boundaries = find_chunk_boundaries(file, desired_num_chunks, split_special_token)
chunks = []

for idx in range(len(chunk_boundaries) - 1):
	file.seek(chunk_boundaries[idx])
	chunks.append(file.read(chunk_boundaries[idx + 1] - chunk_boundaries[idx]))

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

def process_each_chunk(chunk: bytes, special_tokens: list[bytes])-> Counter:
	# Remove special tokens
	pattern = b'|'.join(re.escape(tok) for tok in special_tokens)
	chunk_no_special_tokens = b" ".join(re.split(pattern, chunk))
	counter = Counter()

	for pretoken in re.finditer(PAT, chunk_no_special_tokens.decode("utf-8")):
		pretoken = pretoken.group()
		pretoken = tuple(pretoken.encode("utf-8"))
		counter[tuple(pretoken)] += 1

	return counter

In [46]:
# Run above function on each chunk
with mp.Pool() as pool:
	local_counters = pool.starmap(
		process_each_chunk,
		[(chunk, special_tokens) for chunk in chunks[:20]]
	)

# Merge all local counters
final_counter = Counter()
for counter in local_counters:
	final_counter.update(counter)